In [1]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc

In [2]:
# ---------------------------
# LOAD DATA
# ---------------------------
df = pd.read_csv("D:/Data Analytics/Data Analysis/Clothing Sales Transactions Dataset/Clothing Sales Data_UTF.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   saleID             10000 non-null  object 
 1   saleDate           10000 non-null  object 
 2   productID          10000 non-null  object 
 3   productName        10000 non-null  object 
 4   productCategory    10000 non-null  object 
 5   productColor       10000 non-null  object 
 6   productSize        10000 non-null  object 
 7   quantity           10000 non-null  int64  
 8   unitPrice          10000 non-null  float64
 9   costPrice          10000 non-null  float64
 10  totalAmount        10000 non-null  float64
 11  totalCost          10000 non-null  float64
 12  customerID         10000 non-null  object 
 13  customerName       10000 non-null  object 
 14  location           10000 non-null  object 
 15  locationLatitude   10000 non-null  float64
 16  locationLongitude  1000

In [3]:
# Clean date column
df['saleDate'] = pd.to_datetime(df['saleDate'], errors='coerce')

In [4]:
# ---------------------------
# KPI METRICS
# ---------------------------
total_sales = df["totalAmount"].sum()
total_orders = df.shape[0]
unique_customers = df["customerID"].nunique()
unique_locations = df["location"].nunique()

In [5]:
# ---------------------------
# BAR CHART — PRODUCT CATEGORY SALES
# ---------------------------
category_sales = df.groupby("productCategory")["totalAmount"].sum().reset_index()
fig_category = px.bar(
    category_sales,
    x="productCategory",
    y="totalAmount",
    title="Total Sales by Product Category",
    text_auto=True,
    template="plotly_white"
)

In [6]:
# ---------------------------
# PIE CHART — SALES CHANNEL
# ---------------------------
fig_channel = px.pie(
    df,
    names="salesChannel",
    title="Sales Distribution by Channel",
    hole=0.4
)

In [7]:
# ---------------------------
# MAP — LOCATION BASED SALES
# ---------------------------
location_sales = df.groupby(["location", "locationLatitude", "locationLongitude"])["totalAmount"].sum().reset_index()

fig_map = px.scatter_mapbox(
    location_sales,
    lat="locationLatitude",
    lon="locationLongitude",
    size="totalAmount",
    color="totalAmount",
    hover_name="location",
    zoom=5,
    mapbox_style="open-street-map",
    title="Location-wise Sales"
)

In [8]:
from IPython.display import HTML

HTML("""
<style>

body {
    font-family: 'Inter', 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;
    background-color: #f5f7fa !important;
    color: #333;
}

h1, h2, h3, h4, h5, h6 {
    font-family: 'Inter', 'Segoe UI', Roboto, Helvetica, Arial, sans-serif !important;
    font-weight: 600 !important;
    letter-spacing: 0.5px;
}

.card h4 {
    font-size: 18px;
    font-weight: 500;
    color: #555;
}

.card h2 {
    font-size: 32px;
    font-weight: 700;
    color: #111;
}

.js-plotly-plot .plotly .gtitle {
    font-family: 'Inter', sans-serif !important;
    font-size: 20px !important;
    font-weight: 600 !important;
}

</style>
""")


In [9]:
# DASH APP
app = Dash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = dbc.Container([

    html.H1("Clothing Sales Dashboard", className="text-center mt-3 mb-4"),

    # KPI CARDS ROW
    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H4("Total Sales"),
                html.H2(f"₹ {total_sales:,.2f}")
            ])
        ], className="shadow card"), lg=3, md=6),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H4("Total Orders"),
                html.H2(f"{total_orders}")
            ])
        ], className="shadow card"), lg=3, md=6),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H4("Unique Customers"),
                html.H2(f"{unique_customers}")
            ])
        ], className="shadow card"), lg=3, md=6),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H4("Locations"),
                html.H2(f"{unique_locations}")
            ])
        ], className="shadow card"), lg=3, md=6),
    ], className="mb-4"),

    # ROW 2 — BAR + PIE
    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig_category), lg=8),
        dbc.Col(dcc.Graph(figure=fig_channel), lg=4),
    ], className="mb-4"),

    # ROW 3 — MAP
    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig_map), lg=12),
    ])
], fluid=True, style={"backgroundColor": "#f5f5f5"})


if __name__ == "__main__":
    app.run(mode="inline", debug=True)
